In [1]:
import pickle
import pandas as pd

In [2]:
MODEL_PATH = '../models/model.bin'
RAW_DATA_PATH = '../data/raw/'
INTERIM_DATA_PATH = '../data/interim/'
PROCESSED_DATA_PATH = '../data/processed/'
YEAR = '2021'
MONTH = '02'

In [4]:
with open(MODEL_PATH, 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data(RAW_DATA_PATH + f'fhv_tripdata_{YEAR}-{MONTH}.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [9]:
print(f'Q1. Mean prediction: {y_pred.mean()}')

Q1. Mean prediction: 16.191691679979066


In [10]:
df['ride_id'] = f'{YEAR:.04}/{MONTH:.02}_' + df.index.astype('str')

In [11]:
df['predictions'] = y_pred

In [13]:
df[['ride_id', 'predictions']].to_parquet(
    PROCESSED_DATA_PATH + 'module04_output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)